In [1]:
import pandas as pd
import os 
from tqdm import tqdm 

In [16]:
ASharePriceDf = pd.read_feather("ASharePriceDf.feather")

ASharePriceDf['trade_date'] = pd.to_datetime(ASharePriceDf['trade_date'],format='%Y%m%d')
ASharePriceDf['year'] = ASharePriceDf['trade_date'].dt.year
ASharePriceDf['month'] = ASharePriceDf['trade_date'].dt.month
ASharePriceDf['trade_dt'] = ASharePriceDf.groupby(['ts_code', 'year', 'month'])['trade_date'].transform('count')
ASharePriceDf = ASharePriceDf.query('trade_dt >= 10') #将月交易天数小于10天的股票剔除
ASharePriceDf = ASharePriceDf.query("trade_date <= '2024-09-01'")


In [3]:
tech96_df = pd.read_feather('tech96_df.feather')
tech96_df['trade_date'] = pd.to_datetime(tech96_df['trade_date'])
tech96_df['year'] = tech96_df['trade_date'].dt.year
tech96_df['month'] = tech96_df['trade_date'].dt.month

In [17]:
ASharePriceDf['trade_date'] = pd.to_datetime(ASharePriceDf['trade_date'])
ASharePriceDf['year'] = ASharePriceDf['trade_date'].dt.year
ASharePriceDf['month'] = ASharePriceDf['trade_date'].dt.month
ASharePriceDf = ASharePriceDf.sort_values(by=['ts_code','trade_date'])


In [18]:
ASharePriceDf['ini_price'] = ASharePriceDf.groupby(['ts_code', 'year', 'month'])['close'].transform('first')
ASharePriceDf['end_price'] = ASharePriceDf.groupby(['ts_code', 'year', 'month'])['close'].transform('last')

# 去重并只保留所需的列
monthly_data = ASharePriceDf.drop_duplicates(subset=['ts_code', 'year', 'month'])[['year', 'month', 'ts_code', 'ini_price', 'end_price']]
monthly_data['month_return'] = monthly_data['end_price'] / monthly_data['ini_price'] - 1
monthly_data['next_month_return'] = monthly_data.groupby(['ts_code'])['month_return'].shift(-1)
monthly_data['next_month_return_median'] = monthly_data.groupby(['ts_code'])['next_month_return'].transform('median')
monthly_data['target_label'] = monthly_data['next_month_return'] > monthly_data['next_month_return_median']

monthly_data['target_label'] = monthly_data['target_label'].astype(int)
monthly_data = monthly_data.reindex(columns=['year','month','ts_code','target_label'])
ASharePriceDf = ASharePriceDf.merge(monthly_data,on=['year','month','ts_code'],how='left')
ASharePriceDf = ASharePriceDf.drop(columns=['ini_price','end_price'])

In [6]:
tech96_df = tech96_df.merge(monthly_data,on=['year','month','ts_code'],how='left')


In [7]:
# tech96_df_montly = tech96_df.sort_values(['ts_code', 'trade_date'])
# tech96_df_montly = tech96_df_montly.groupby(['ts_code', 'year', 'month']).last().reset_index()

In [8]:
# tech96_df_montly = pd.read_feather("tech96_df_only_keep_monthly_data.feather")

In [30]:
# tech96_df_montly = tech96_df_montly.loc[tech96_df_montly['target_label'].notna()]

In [10]:
import os 
base_dir = 'monthly_data'
if not os.path.exists(base_dir):
   os.makedirs(base_dir)

for (year, month), group_df in tqdm(tech96_df.groupby(['year', 'month'])):
   # 创建文件名
   filename = f"{year:04d}_{month:02d}.feather"
   filepath = os.path.join(base_dir, filename)
   
   # 保存为feather格式
   group_df.reset_index(drop=True).to_feather(filepath)

### 随机森林

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
# from cuml.ensemble import RandomForestClassifier
import os
import pandas as pd
from tqdm import tqdm

In [5]:
# # 初始化随机森林模型
# rf_model = RandomForestClassifier(
#     n_estimators=0,  # 初始树的数量为 0
#     max_depth=23,
#     random_state=42,
#     warm_start=True  # 启用增量训练
# )

# # 每批次添加一定数量的树
# trees_per_batch = 10  # 每批次添加的树数量
# base_dir = 'monthly_data'

# for filename in tqdm(os.listdir(base_dir)):
#     filepath = os.path.join(base_dir, filename)
#     batch = pd.read_feather(filepath)
#     batch.replace([np.inf, -np.inf], np.nan, inplace=True)
#     batch = batch.groupby('ts_code').ffill()
#     # 用均值填充 NaN
#     batch = batch.dropna()
#     feature_columns = [col for col in batch.columns if col not in ['trade_date','ts_code', 'year', 'month', 'target_label']]
#     X_batch = batch[feature_columns]
#     y_batch = batch['target_label']

#     # 更新模型
#     rf_model.n_estimators += trees_per_batch
#     rf_model.fit(X_batch, y_batch)



In [14]:
tech96_df_montly = pd.read_feather("tech96_df_only_keep_monthly_data.feather")

In [19]:
tech96_df_montly = tech96_df_montly.merge(monthly_data,on=['year','month','ts_code'],how='left')


In [21]:
tech96_df_montly.replace([np.inf, -np.inf], np.nan, inplace=True)

# 用均值填充 NaN
tech96_df_montly = tech96_df_montly.dropna()
feature_columns = [col for col in tech96_df_montly.columns if col not in ['ts_code', 'year', 'month', 'target_label']]
X = tech96_df_montly[feature_columns]  # 技术指标作为特征
y = tech96_df_montly['target_label']    # 标签列

# 分割训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, max_depth=23, random_state=42, verbose=1, n_jobs=-1)

# 训练模型
rf_model.fit(X_train, y_train)

# 在测试集上验证
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

tech96_df_montly['outperformance_prob'] = rf_model.predict_proba(X)[:, 1] 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   25.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.4min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.7s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    2.7s finished
C:\Users\ASUS-ROG\AppData\Local\Temp\ipykernel_33136\2076736170.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

In [23]:
tech96_df_montly

,ts_code,year,month,Avg50,Avg100,Avg200,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,...,sharpe_ratio,skewness,kurtosis,max_spread,avg_return,short_vola,long_vola,difference_vola,target_label,outperformance_prob
13,000001.SZ,2011,4,0.146096,0.088517,-0.093625,0.516287,17.8015,19.307120,16.295880,...,-0.406461,-0.227883,1.999525,0.149340,-0.000539,0.016480,0.021061,-0.004580,0.0,0.308717
14,000001.SZ,2011,5,0.086769,0.065782,-0.024848,0.540419,17.9305,18.631770,17.229230,...,-0.453489,-0.276741,1.941576,0.149340,-0.000599,0.018091,0.020965,-0.002874,0.0,0.283594
15,000001.SZ,2011,6,0.000586,0.126733,-0.002338,0.486306,16.5475,17.256834,15.838166,...,-0.707433,-0.267984,2.006079,0.149340,-0.000931,0.019154,0.020892,-0.001739,1.0,0.449908
16,000001.SZ,2011,7,-0.042518,0.088512,0.041441,0.491620,17.3695,17.951856,16.787144,...,-0.259283,-0.029276,1.320346,0.137029,-0.000331,0.017216,0.020279,-0.003063,0.0,0.282398
17,000001.SZ,2011,8,0.035012,0.028694,-0.092623,0.516062,16.6585,17.428754,15.888246,...,-0.077453,0.323796,1.189958,0.129440,-0.000094,0.019355,0.019321,0.000034,0.0,0.307297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582121,689009.SH,2024,4,0.276063,0.118425,0.010867,1.790534,30.8310,37.104171,24.557829,...,0.215422,1.406040,9.241159,0.288173,0.000393,0.036446,0.028948,0.007498,1.0,0.667730
582122,689009.SH,2024,5,0.079114,0.298373,0.080980,1.873139,39.3015,43.186303,35.416697,...,0.381659,1.414143,8.836023,0.288173,0.000703,0.033643,0.029231,0.004412,0.0,0.407685
582123,689009.SH,2024,6,0.252041,0.514192,0.176790,1.874123,38.9765,42.040512,35.912488,...,0.168117,0.250690,1.523486,0.197764,0.000286,0.032494,0.026994,0.005500,1.0,0.429382
582124,689009.SH,2024,7,0.022710,0.365090,0.176572,1.849665,38.2560,40.385990,36.126010,...,0.583742,0.198067,1.407309,0.197764,0.001000,0.027153,0.027203,-0.000050,1.0,0.655131


In [24]:
tech96_df_montly.to_feather("forecast_result_monthly.feather")
